In [60]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
# Requests
import requests
from time import sleep
from tqdm import tqdm

# Para convertir de string a diccionario
import ast
import os
import json

import sys
sys.path.append("../")
import src.soporte_spotify as api

In [61]:
# sp = api.load_credentials()
# sp.current_user()

In [62]:
zara = pd.read_csv('../datos/01 Spotify/01_followers_zara.csv')

In [63]:
zara.sample()

,brand,username,user_id,playlists,playlist_ids,playlist_ids_limited,artistas
444,Zara,Ksenya,31w3ckixld5d37zkjobvwko2c35e,{'the start of new period ': '1dUhimPQKl4eugbl...,"['1dUhimPQKl4eugblaUhKMf', '0mJQKF1hxAhPP3UddJ...","['1dUhimPQKl4eugblaUhKMf', '0mJQKF1hxAhPP3UddJ...","{""1mXqOPpMYngJYilH8L1ZDG"": ""The Real Btx"", ""3e..."


### Eliminar aquellos users cuyas playlists no tenían artistas (no había canciones)

In [64]:
zara = zara.loc[zara["artistas"] != "{}"]

### Reemplazar null por None para poder convertirlo a diccionarios

In [75]:
for idx in zara.index:
    zara.loc[idx,"artistas"] = zara.loc[idx,"artistas"].replace("null","None")

### Convertir:
- playlists: de STR a dict
- playlist_ids: de STR a list
- playlist_ids_limited: de STR a list
- artistas: de STR a dict

In [77]:
zara["playlists"] = zara["playlists"].apply(ast.literal_eval)
zara["playlist_ids"] = zara["playlist_ids"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
zara["playlist_ids_limited"] = zara["playlist_ids_limited"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
zara["artistas"] = zara["artistas"].apply(ast.literal_eval)
zara.to_csv("../datos/01 Spotify/01_followers_zara.csv",index=False)

### Ver cuantos artistas tenemos en total

In [83]:
suma = 0
for artista in zara["artistas"]:
    suma += len(artista)

suma

323870

# Diccionario de artistas únicos
- Creamos un diccionario de artistas únicos a partir de todos los obtenidos
- De forma que solo consultaremos los géneros una sola vez

Diccionario Nº Veces que aparece un artista
- de los usuarios, obtendremos el número de veces que sale un artista
- De esta manera sabremos la popularidad dentro de los seguidores de Zara
